In [1]:
%matplotlib inline

import os, sys, glob, uuid
import numpy as np
import matplotlib.pyplot as pl

from scipy.integrate import ode
from IPython.display import Image, display


/Users/Olivier/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## One-body problem computed and displayed as animated .gif

In [2]:
%%time

# Compute RHS of ODE
def f(t,Y):
    x, y, vx, vy = Y # define individual values
    d3 = (x**2 + y**2)**1.5 # ||x||^3
    M = 0.5
    return np.array([vx,vy, -M*x/d3, -M*y/d3])

# init params
t0 = 0 # initial start time
tfinal = .43 # final start time
dt = .002 # time step to solution (dopri5 algorithm uses adaptive)
y0 = np.array([.25, 0.0, 0.0, .45]) # initial position and velocity (x, y, vx, vy)

# init integrator object
test = ode(f).set_integrator('dopri5', atol=1e-6) # prescribe tolerance for adaptive time step
test.set_initial_value(y0, t0) # set initial time and initial value

# init matplotlib
fig = pl.figure(figsize=(12, 6))
ax = fig.add_subplot(111)

# init frames
img_folder = 'img'
img_fn = []
img_radical = '_img'
img_no = []
img_count = 0
if not os.path.exists(img_folder):
    os.makedirs(img_folder)
else:
    for f in os.listdir(img_folder):
        os.remove(os.path.join(img_folder, f))
    
print 'Creating images...'
while test.successful() and test.t < tfinal:
    # integrate
    test.integrate(test.t+dt)

    # plot position
    ax.cla()
    ax.plot(test.y[0], test.y[1], 'b.', markersize=9)
    ax.plot(0, 0, 'oy', markersize=15)
    ax.set_xlim([-0.1, 0.3])
    ax.set_ylim([-0.15, 0.15])
    ax.set_title('One-body orbit t=%2.3f' % (test.t))

    # save image
    img_fn.append(os.path.join('img', img_radical+'%03d'%img_count+'.png'))
    img_no.append(img_count)
    img_count += 1
    
    if img_count%5==0:
        sys.stdout.write('\r')
        sys.stdout.write('%i / %i' %(img_count, tfinal/dt))
        sys.stdout.flush()
    
    fig.savefig(img_fn[-1]);

pl.close(fig);

Creating images...
215 / 215CPU times: user 18.6 s, sys: 172 ms, total: 18.8 s
Wall time: 18.8 s


In [3]:
%%time

# convert images to animated gif
print '\nConvert png files to animated gif...'

def make_gif(folder, output, delay=100, repeat=True):
    """Uses imagemagick/convert to produce an animated .gif from a list of picture files"""
    loop = -1 if repeat else 0
    input_files = folder+'/*.png'
    print 'convert -delay {} -loop {} {} {}'.format(delay, loop, input_files, output)
    os.system('convert -delay {} -loop {} {} {}'.format(delay, loop, input_files, output))
    
for f in glob.glob('orbit*.gif'):
    os.remove(f)
img_id = uuid.uuid4()
make_gif(img_folder, 'orbit%s.gif' % img_id, delay=4, repeat=False)

# display animated gif
display(Image(url='orbit%s.gif' % img_id))


Convert png files to animated gif...
convert -delay 4 -loop 0 img/*.png orbit1947e978-51ad-4ff6-b983-754f42584df1.gif


CPU times: user 2.17 ms, sys: 841 µs, total: 3.01 ms
Wall time: 5.87 s
